In [1]:
import numpy as np
import pandas as pd
import time
import multiprocessing
from multiprocessing import Pool
import functools
import sys
from pfsm import *

In [2]:
pd.set_option('display.max_colwidth', -1)
dtf_names =  pd.read_csv('data/sec_edgar_company_info.csv', dtype = object)
print(dtf_names.shape)
dtf_names.head()

(663000, 3)


,Line Number,Company Name,Company CIK Key
0,1,!J INC,1438823
1,2,"#1 A LIFESAFER HOLDINGS, INC.",1509607
2,3,#1 ARIZONA DISCOUNT PROPERTIES LLC,1457512
3,4,#1 PAINTBALL CORP,1433777
4,5,$ LLC,1427189


In [3]:
time_start = time.time()
str_strings = dtf_names["Company Name"].values
M = tfidfM(str_strings)
print(M.shape)
print(time.time() - time_start)

(663000, 27922)
13.962115049362183


In [4]:
# time_start = time.time()
# The following code runs the optimized cosine similarity function. 
# It only stores the top 10 most similar items, and only items with a similarity above 0.8
# matches = awesome_cossim_top(M, M.transpose(), 10, 0.8)
# print(time.time() - time_start)

In [5]:
def myf(idxs, ns):
    min_idxs = min(idxs)
    matches_tmp = awesome_cossim_top(ns.M[idxs, ], ns.M[min_idxs:,].transpose(), 10, 0.8)
    return [matches_tmp[j, ].indices + min_idxs for j in range(matches_tmp.shape[0])]

In [6]:
time_start = time.time()
int_entity_id = np.array([np.NaN for j in range(len(str_strings))])
lst_split = list(split(range(M.shape[0]), 100))
print(time.time() - time_start)

0.08235836029052734


In [7]:
time_start = time.time()
manager = multiprocessing.Manager()
ns = manager.Namespace()
ns.M = M
print(time.time() - time_start)

0.6267702579498291


In [8]:
time_start = time.time()
pool = Pool(6)
r = pool.map(functools.partial(myf, ns = ns), lst_split)
pool.close()
pool.join()
print(time.time() - time_start)

644.5067074298859


In [9]:
time_start = time.time()


out = np.array([])
for j in range(len(r)):
    out = np.append(out, r[j])

out
r = out

    
for j in range(len(r)):
    idx = r[j]
    if np.isscalar(idx):
        idx = np.array([idx])
    idx = idx.astype(int)
    tmp_idx = int_entity_id[idx]
    if sum([np.isnan(x) for x in tmp_idx]) == len(tmp_idx):
        int_entity_id[idx] = j
    else:
        int_entity_id[idx] = np.nanmin(tmp_idx)


dtf_out = pd.DataFrame({"STRING": str_strings, "ENTITY_ID": int_entity_id}).sort_values(["ENTITY_ID"])


print(time.time() - time_start)

16.10629439353943


In [10]:
dtf_out.head(100)

,STRING,ENTITY_ID
0,!J INC,0.0
1,"#1 A LIFESAFER HOLDINGS, INC.",1.0
2,#1 ARIZONA DISCOUNT PROPERTIES LLC,2.0
3,#1 PAINTBALL CORP,3.0
4,$ LLC,4.0
5,& S MEDIA GROUP LLC,5.0
278210,HH & S MEDIA GROUP LLC,5.0
6,&TV COMMUNICATIONS INC.,6.0
7,"'MKTG, INC.'",7.0
8,'OHANA LABS INC.,8.0
